Build and Configure an Azure VM with FsCloudInit and Farmer
========

Azure Virtual Machines start with just a base operating system image, but you will typically need additional packages installed and configured. Cloud-init is a standard technology used across most public clouds to initialize VMs, and even if you don't use cloud-init directly, it is used by the cloud provider to create initial users and add authorized SSH public keys. You can also define your own cloud-init configuration to add additional package sources and packages, create files, and run bootstrapping commands to get your machine fully initialized.

[FsCloudInit](https://github.com/ninjarobot/FsCloudInit) is a library to help build cloud-init configuration files using a Domain Specific Language (DSL) that helps with writing the specifications and preventing common errors. [Farmer](https://github.com/CompositionalIT/farmer) is another library for authoring Azure Resource Manager (ARM) deployment templates. These two libraries compose well together to enable you to write an ARM deployment specification to deploy a VM that will run a cloud-init configuration to configure it as needed.

We will build a cloud-init spec and an ARM template using FsCloudInit and Farmer. This will be used to provision a virtual machine, install the .NET SDK, generate an ASP.NET Core MVC app, enable it to run as a service, and start it.

First, the two packages are needed and we will open some namespaces that we need.

In [ ]:
#r "nuget:FsCloudInit"
#r "nuget:Farmer"

open System
open System.IO
open System.Net.Http
open FsCloudInit
open FsCloudInit.Builders
open Farmer
open Farmer.Builders

Two files are needed on the VM:

* buildapp.sh - normally we will be deploying a package from our CI system, but for the purposes of this demo, we'll generate an application using a `dotnet new` template.
* myapp.service - this is a `systemd` service for our application so it will run on VM startup, will be restarted if it crashes, and allows us to specify the user.

#### buildapp.sh
```bash
#!/bin/bash
set -eux

mkdir -p /home/azureuser/app
chown -R azureuser:azureuser /home/azureuser
sudo -u azureuser sh -c 'cd /home/azureuser/app && dotnet new mvc'
```

#### myapp.service
```
[Unit]
Description=My mvc app
StartLimitIntervalSec=30s
StartLimitBurst=3

[Service]
ExecStart=/usr/bin/dotnet run --urls http://*:8080 --environment Production
User=azureuser
WorkingDirectory=/home/azureuser/app
Restart=on-failure
RestartSec=5s
TimeoutSec=100s

[Install]
WantedBy=multi-user.target
```

Next, we need to generate our cloud-init configuration file.

In [ ]:
let userData =
    // Get the Microsoft apt package source and published gpg key
    let aptSourceValue, gpgKey =
        task {
            use http = new HttpClient ()
            let! aptSourceRes = http.GetAsync "https://packages.microsoft.com/config/ubuntu/20.04/prod.list"
            let! aptSourceVal = aptSourceRes.Content.ReadAsStringAsync ()
            let! gpgKeyRes = http.GetAsync "https://packages.microsoft.com/keys/microsoft.asc"
            let! gpgKey = gpgKeyRes.Content.ReadAsStringAsync ()
            return aptSourceVal, gpgKey
        } |> Async.AwaitTask |> Async.RunSynchronously

    // Normally we will clone a repo or install a package with our application.
    let bootstrapScript = "/home/azureuser/buildapp.sh"

    // Define the cloud init configuration for the VM
    cloudConfig {
        add_apt_sources [
            aptSource {
                name "microsoft-prod"
                key gpgKey
                source aptSourceValue
            }
        ]
        package_update true
        add_packages [
            Package "apt-transport-https"
            PackageVersion (PackageName="dotnet-sdk-6.0", PackageVersion="6.0.201-1")
        ]
        write_files [
            writeFile { // Embeds our buildapp.sh script to generate the application.
                path bootstrapScript
                permissions "0764"
                content ( "buildapp.sh" |> File.ReadAllText )
            }
            writeFile { // Embeds our systemd service.
                path "/lib/systemd/system/myapp.service"
                content ( "myapp.service" |> File.ReadAllText )
            }
        ]
        run_commands [
            // Run the bootstrap script
            [ bootstrapScript ]
            // Enable the 'myapp' systemd service on startup
            [ "systemctl"; "enable"; "myapp" ]
            // Start the 'myapp' systemd service now (see 'journalctl -f -u myapp' for logs).
            [ "systemctl"; "start"; "myapp" ]
        ]
    }
    |> Writer.write

### Cloud-init Specification
You can see the `userData` contains a valid cloud-init specification. 

In [ ]:
userData

We will include this `userData` in the VM resource properties in an ARM deployment template so that when Azure provisions the VM, it runs our additional configuration steps.

In [ ]:
// Instead of using a password, we will provide our public SSH key to enable us to connect to the VM.
let mySshPubKey =
    [|
        Environment.GetFolderPath Environment.SpecialFolder.UserProfile
        ".ssh"
        "id_rsa.pub"
    |]
    |> Path.Combine
    |> File.ReadAllText

// We will use an Ubuntu image since we are using `apt` in our cloud-init configuration.
let UbuntuServer_2004LTS = Vm.makeLinuxVm "0001-com-ubuntu-server-focal" "canonical" "20_04-lts-gen2" 

// Build the deployment template.
let vmDeployment =
    arm {
        location Location.EastUS
        add_resources [
            vm {
                name "my-app-server"
                username "azureuser"
                vm_size Vm.Standard_B1ms
                operating_system UbuntuServer_2004LTS
                diagnostics_support
                disable_password_authentication true
                // Adds our public key.
                add_authorized_key "/home/azureuser/.ssh/authorized_keys" mySshPubKey
                // Embeds our cloud-init spec as a base64 string in the VM resource.
                custom_data userData
                // Could use a bigger VM as a spot instance to keep costs down.
                // vm_size (Vm.CustomImage "Standard_D2_v5")
                // spot_instance (Vm.Deallocate, 0.0285M)
            }
        ]
    }

### ARM Template
View the resulting ARM deployment template.

In [ ]:
vmDeployment.Template |> Writer.toJson

### Summary

With FsCloudInit and Farmer, we can use a high-level language to build cloud-init and ARM deployment specifications. As our needs change or complexity grows over time, we are able to continue to generate these specifications reliably. The above specification can be deployed via the Azure Portal, the CLI with `az deployment group create` or with Farmer using `Deploy.execute`. 